In [3]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import time
import ast

/Users/vitormafra/Desktop/hackathon-elevenlabs/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
df_user_reviews = pd.read_csv("../data/RAW_interactions.csv")

In [14]:
df_user_reviews

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."
...,...,...,...,...,...
1132362,116593,72730,2003-12-09,0,Another approach is to start making sauce with...
1132363,583662,386618,2009-09-29,5,These were so delicious! My husband and I tru...
1132364,157126,78003,2008-06-23,5,WOW! Sometimes I don't take the time to rate ...
1132365,53932,78003,2009-01-11,4,Very good! I used regular port as well. The ...


In [15]:
df_user_reviews.describe()

,user_id,recipe_id,rating
count,1.132367e+06,1.132367e+06,1.132367e+06
mean,1.384291e+08,1.608972e+05,4.411016e+00
std,5.014269e+08,1.303987e+05,1.264752e+00
min,1.533000e+03,3.800000e+01,0.000000e+00
25%,1.354700e+05,5.425700e+04,4.000000e+00
50%,3.309370e+05,1.205470e+05,5.000000e+00
75%,8.045500e+05,2.438520e+05,5.000000e+00
max,2.002373e+09,5.377160e+05,5.000000e+00


In [16]:
df_recipes = pd.read_csv("../data/RAW_recipes.csv")

In [17]:
df_recipes

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
...,...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22
231633,zydeco spice mix,493372,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13
231634,zydeco ya ya deviled eggs,308080,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8
231635,cookies by design cookies on a stick,298512,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10


In [18]:
df_recipes.dropna()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
...,...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22
231633,zydeco spice mix,493372,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13
231634,zydeco ya ya deviled eggs,308080,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8
231635,cookies by design cookies on a stick,298512,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10


In [19]:
def get_best_recipes(df_recipes, df_user_reviews, min_rating=3.9):
    """
    Filters the best recipes based on average user ratings and includes all comments.
    
    Parameters:
    df_recipes (pandas.DataFrame): DataFrame with recipe information
    df_user_reviews (pandas.DataFrame): DataFrame with user reviews
    min_rating (float): Minimum average rating to filter recipes (default: 3.9)
    
    Returns:
    pandas.DataFrame: DataFrame with best recipes, their average ratings, and all comments
    """
    # Calculate average ratings per recipe
    avg_ratings = df_user_reviews.groupby('recipe_id')['rating'].agg([
        'mean',
        'count'
    ]).reset_index()
    
    # Rename columns for clarity
    avg_ratings.columns = ['id', 'rating_mean', 'rating_count']
    
    # Aggregate all comments for each recipe into a list
    comments_by_recipe = df_user_reviews.groupby('recipe_id')['review'].agg(list).reset_index()
    comments_by_recipe.columns = ['id', 'all_reviews']
    
    # Merge rating information with recipe data
    recipes_with_ratings = pd.merge(
        df_recipes,
        avg_ratings,
        on='id',
        how='inner'
    )
    
    # Merge comments with the recipe data
    recipes_with_ratings = pd.merge(
        recipes_with_ratings,
        comments_by_recipe,
        on='id',
        how='left'
    )
    
    # Filter only recipes with average rating higher than min_rating
    best_recipes = recipes_with_ratings[recipes_with_ratings['rating_mean'] >= min_rating]
    
    # Sort by average rating (best first)
    best_recipes = best_recipes.sort_values('rating_mean', ascending=False)
    
    return best_recipes

In [20]:
df_best_recipes = get_best_recipes(df_recipes, df_user_reviews)
df_best_recipes

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,rating_mean,rating_count,all_reviews
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,5.0,3,[ I used an acorn squash and recipe#137681 Swe...
130923,marinated greek feta,141627,5,197023,2005-10-17,"['15-minutes-or-less', 'time-to-make', 'course...","[603.9, 93.0, 10.0, 48.0, 19.0, 79.0, 2.0]",4,['cut the feta cheese into 2cm cubes and place...,"i found this simple, super-quick-to-make recip...","['greek feta cheese', 'olive oil', 'cracked bl...",7,5.0,8,[Just delicious and a wonderful way to keep my...
130921,marinated goat cheese with fresh oregano,63342,65,56081,2003-05-28,"['weeknight', 'time-to-make', 'course', 'main-...","[130.1, 17.0, 1.0, 5.0, 12.0, 27.0, 0.0]",2,"['place oil , oregano and pepper in a narrow s...",another one from the atkins low carb cook book...,"['extra virgin olive oil', 'oregano leaves', '...",4,5.0,1,[This was really good! I wasn't sure if I lik...
130920,marinated goat cheese spread dip,258115,195,444132,2007-10-10,"['time-to-make', 'course', 'main-ingredient', ...","[136.8, 18.0, 1.0, 7.0, 11.0, 24.0, 0.0]",6,['arrange cheese in a shallow dish in a single...,a decedent spread that wins raves at potlucks ...,"['soft fresh goat cheese', 'kalamata olive', '...",10,5.0,3,"[I'm not a big fan of goat cheese, but made th..."
130917,marinated goat cheese,56514,2885,62043,2003-03-18,"['weeknight', 'time-to-make', 'course', 'main-...","[916.0, 145.0, 9.0, 16.0, 32.0, 127.0, 1.0]",4,['cut goat cheese into 1 cm rounds and place i...,this recipe is from the ottawa citizen. this i...,"['chevre cheese', 'dried chili', 'whole black ...",9,5.0,2,[This is a delicious appetizer to have on hand...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132386,mean green juice for juicer,492690,17,442988,2012-12-31,"['30-minutes-or-less', 'time-to-make', 'course...","[136.6, 1.0, 93.0, 2.0, 4.0, 0.0, 11.0]",2,"['wash all ingredients thoroughly', 'run throu...","this is the recipe for ""mean green"" juice used...","['cucumber', 'celery ribs', 'apples', 'kale le...",6,3.9,10,[DH and I are juicing and this is a great go t...
215465,tony roma s blue ridge smokies sauce by todd w...,2576,50,1534,1999-10-25,"['60-minutes-or-less', 'time-to-make', 'course...","[634.2, 0.0, 557.0, 108.0, 6.0, 0.0, 52.0]",3,['combine all of the ingredients in a medium s...,from www.topsecretrecipes.com,"['ketchup', 'apple cider vinegar', 'brown suga...",9,3.9,10,"[Excellent sauce. A little sweet, a little bi..."
62781,creamy cocoa mix extra large batch,107668,15,154044,2005-01-05,"['15-minutes-or-less', 'time-to-make', 'course...","[66.5, 4.0, 31.0, 1.0, 3.0, 11.0, 2.0]",3,['mix all ingredients in a large bowl and blen...,"i like this cocoa for many reasons, only one o...","['unsweetened cocoa powder', 'powdered sugar',...",5,3.9,10,"[I am feeling under-the-weather today, and nee..."
45886,chicken strips with sweet and sour sauce,19266,30,27416,2002-02-09,"['30-minutes-or-less', 'time-to-make', 'course...","[451.2, 18.0, 153.0, 35.0, 62.0, 15.0, 17.0]",20,['remove all fat from chicken breasts and disc...,a fantastic chicken recipe with a delightful s...,"['boneless skinless chicken breasts', 'eggs', ...",20,3.9,10,[My family really enjoyed these chicken strip...


In [21]:
recipe_example = df_best_recipes[['name', 'rating_mean', 'rating_count', 'all_reviews']].iloc[0]
print(f"Recipe: {recipe_example['name']}")
print(f"Average Rating: {recipe_example['rating_mean']:.2f}")
print(f"Number of ratings: {recipe_example['rating_count']}")
print("\nSample of reviews:")
for review in recipe_example['all_reviews'][:3]:  # Show first 3 reviews
    print(f"- {review}")

Recipe: arriba   baked winter squash mexican style
Average Rating: 5.00
Number of ratings: 3

Sample of reviews:
-  I used an acorn squash and recipe#137681 Sweet Mexican spice blend. Only used 1 tsp honey & 1 tsp butter between both halves,, sprinkled the squash liberally with the spice mix. Baked covered for 45 minutes uncovered or 15.  I basted the squash   with the the butter/honey from the cavity  allowing it to get a golden color.  Lovely Squash recipe Thanks Cookgirl
- This was a nice change. I used butternut squash and the sweet option using a good local honey and unsalted butter. I did not add salt. We ate this on top of recipe#322603 with Balkan yogurt. I may make this again same option. Made for Ramadan Tag 2010.
- Excellent recipe! I used butternut squash and the sweet option. The mexican spice mix put this over the top. Thanks for sharing.


In [22]:
df_best_recipes["tags"].unique()

array(["['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']",
       "['15-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'south-west-pacific', 'appetizers', 'lunch', 'eggs-dairy', 'australian', 'greek', 'easy', 'european', 'no-cook', 'beginner-cook', 'potluck', 'dinner-party', 'finger-food', 'picnic', 'vegetarian', 'cheese', 'dietary', 'gluten-free', 'high-calcium', 'low-carb', 'egg-free', 'free-of-something', 'high-in-something', 'low-in-something', 'taste-mood', 'savory', 'to-go', 'presentation', 'served-cold', '3-steps-or-less', 'technique']",
       "['weeknight', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', '5-ingredients-or-less', 'appetizers', 'eggs-dairy', 'easy', 'european',

In [23]:
df_best_recipes["tags"].unique()[0]

"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']"

In [24]:
df_best_recipes[df_best_recipes["name"] == "boeuf bourguignon"]

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,rating_mean,rating_count,all_reviews
26655,boeuf bourguignon,372650,190,461834,2009-05-18,"['time-to-make', 'preparation', '4-hours-or-le...","[2233.4, 303.0, 73.0, 40.0, 63.0, 385.0, 16.0]",19,['marinate the beef for at least 4 hours in th...,boeuf bourgignon comes from the bourgignon reg...,"['beef', 'red wine', 'onion', 'garlic cloves',...",13,4.6,5,[This dish was very tasty! It has so much fla...


In [25]:
def split_nutrition_column(df):
    """
    Splits the nutrition column into separate columns for each nutritional value.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing the nutrition column
    
    Returns:
    pandas.DataFrame: DataFrame with separated nutrition columns
    """
    # Create a copy of the DataFrame to avoid modifying the original
    df_new = df.copy()
    
    # If the nutrition column contains strings (like '[2233.4, 303.0, ...]'),
    # convert them to actual lists
    if isinstance(df_new['nutrition'].iloc[0], str):
        df_new['nutrition'] = df_new['nutrition'].apply(ast.literal_eval)
    
    # Create new columns from the nutrition list
    nutrition_df = pd.DataFrame(df_new['nutrition'].tolist(), columns=[
        'calories',
        'total_fat_pdv',
        'sugar_pdv',
        'sodium_pdv',
        'protein_pdv',
        'saturated_fat_pdv',
        'carbohydrates_pdv'
    ])
    
    # Drop the original nutrition column and join the new columns
    df_new = df_new.drop('nutrition', axis=1)
    df_new = pd.concat([df_new, nutrition_df], axis=1)
    
    return df_new

In [26]:
df_recipes_expanded = split_nutrition_column(df_best_recipes)

In [27]:
df_recipes_expanded

,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,...,rating_mean,rating_count,all_reviews,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbohydrates_pdv
0,arriba baked winter squash mexican style,137739.0,55.0,47892.0,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...",11.0,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",...,5.0,3.0,[ I used an acorn squash and recipe#137681 Swe...,51.5,0.0,13.0,0.0,2.0,0.0,4.0
130923,marinated greek feta,141627.0,5.0,197023.0,2005-10-17,"['15-minutes-or-less', 'time-to-make', 'course...",4.0,['cut the feta cheese into 2cm cubes and place...,"i found this simple, super-quick-to-make recip...","['greek feta cheese', 'olive oil', 'cracked bl...",...,5.0,8.0,[Just delicious and a wonderful way to keep my...,280.9,18.0,39.0,23.0,36.0,22.0,9.0
130921,marinated goat cheese with fresh oregano,63342.0,65.0,56081.0,2003-05-28,"['weeknight', 'time-to-make', 'course', 'main-...",2.0,"['place oil , oregano and pepper in a narrow s...",another one from the atkins low carb cook book...,"['extra virgin olive oil', 'oregano leaves', '...",...,5.0,1.0,[This was really good! I wasn't sure if I lik...,284.7,16.0,10.0,24.0,46.0,9.0,7.0
130920,marinated goat cheese spread dip,258115.0,195.0,444132.0,2007-10-10,"['time-to-make', 'course', 'main-ingredient', ...",6.0,['arrange cheese in a shallow dish in a single...,a decedent spread that wins raves at potlucks ...,"['soft fresh goat cheese', 'kalamata olive', '...",...,5.0,3.0,"[I'm not a big fan of goat cheese, but made th...",555.9,58.0,13.0,56.0,83.0,73.0,4.0
130917,marinated goat cheese,56514.0,2885.0,62043.0,2003-03-18,"['weeknight', 'time-to-make', 'course', 'main-...",4.0,['cut goat cheese into 1 cm rounds and place i...,this recipe is from the ottawa citizen. this i...,"['chevre cheese', 'dried chili', 'whole black ...",...,5.0,2.0,[This is a delicious appetizer to have on hand...,325.6,13.0,10.0,27.0,13.0,27.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,633.1,41.0,13.0,66.0,56.0,79.0,23.0
195161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,198.0,4.0,19.0,21.0,13.0,3.0,12.0
195162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,49.1,5.0,18.0,0.0,0.0,8.0,1.0
195163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,80.3,7.0,16.0,2.0,2.0,19.0,2.0


In [28]:
df_recipes_expanded.dropna()

,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,...,rating_mean,rating_count,all_reviews,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbohydrates_pdv
0,arriba baked winter squash mexican style,137739.0,55.0,47892.0,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...",11.0,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",...,5.0,3.0,[ I used an acorn squash and recipe#137681 Swe...,51.5,0.0,13.0,0.0,2.0,0.0,4.0
130923,marinated greek feta,141627.0,5.0,197023.0,2005-10-17,"['15-minutes-or-less', 'time-to-make', 'course...",4.0,['cut the feta cheese into 2cm cubes and place...,"i found this simple, super-quick-to-make recip...","['greek feta cheese', 'olive oil', 'cracked bl...",...,5.0,8.0,[Just delicious and a wonderful way to keep my...,280.9,18.0,39.0,23.0,36.0,22.0,9.0
130921,marinated goat cheese with fresh oregano,63342.0,65.0,56081.0,2003-05-28,"['weeknight', 'time-to-make', 'course', 'main-...",2.0,"['place oil , oregano and pepper in a narrow s...",another one from the atkins low carb cook book...,"['extra virgin olive oil', 'oregano leaves', '...",...,5.0,1.0,[This was really good! I wasn't sure if I lik...,284.7,16.0,10.0,24.0,46.0,9.0,7.0
130920,marinated goat cheese spread dip,258115.0,195.0,444132.0,2007-10-10,"['time-to-make', 'course', 'main-ingredient', ...",6.0,['arrange cheese in a shallow dish in a single...,a decedent spread that wins raves at potlucks ...,"['soft fresh goat cheese', 'kalamata olive', '...",...,5.0,3.0,"[I'm not a big fan of goat cheese, but made th...",555.9,58.0,13.0,56.0,83.0,73.0,4.0
130917,marinated goat cheese,56514.0,2885.0,62043.0,2003-03-18,"['weeknight', 'time-to-make', 'course', 'main-...",4.0,['cut goat cheese into 1 cm rounds and place i...,this recipe is from the ottawa citizen. this i...,"['chevre cheese', 'dried chili', 'whole black ...",...,5.0,2.0,[This is a delicious appetizer to have on hand...,325.6,13.0,10.0,27.0,13.0,27.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2673,affordable basic restaurant style tomato and m...,115199.0,15.0,118159.0,2005-04-04,"['15-minutes-or-less', 'time-to-make', 'course...",6.0,"['in a medium saucepan', 'bring salted water ,...",ever had this comfort soup in a restaurant? t...,"['tomato juice', 'water', 'onion', 'macaroni',...",...,3.9,10.0,"[hello Andre, I have finally come on to rate ...",291.2,26.0,73.0,7.0,13.0,59.0,9.0
174846,roast pork pernil puerto rican style,316839.0,255.0,812387.0,2008-07-31,"['main-ingredient', 'cuisine', 'preparation', ...",10.0,['the night before peel garlic and with a piln...,"tasty roast pork, puerto rican style, marine o...","['pork shoulder', 'corn oil', 'white vinegar',...",...,3.9,10.0,[Foolproof recipe for making a flavorful and m...,1980.0,301.0,8.0,172.0,70.0,591.0,5.0
132386,mean green juice for juicer,492690.0,17.0,442988.0,2012-12-31,"['30-minutes-or-less', 'time-to-make', 'course...",2.0,"['wash all ingredients thoroughly', 'run throu...","this is the recipe for ""mean green"" juice used...","['cucumber', 'celery ribs', 'apples', 'kale le...",...,3.9,10.0,[DH and I are juicing and this is a great go t...,499.2,32.0,17.0,28.0,37.0,61.0,20.0
62781,creamy cocoa mix extra large batch,107668.0,15.0,154044.0,2005-01-05,"['15-minutes-or-less', 'time-to-make', 'course...",3.0,['mix all ingredients in a large bowl and blen...,"i like this cocoa for many reasons, only one o...","['unsweetened cocoa powder', 'powdered sugar',...",...,3.9,10.0,"[I am feeling under-the-weather today, and nee...",1511.8,140.0,210.0,232.0,177.0,151.0,26.0


In [29]:
df_recipes_expanded = df_recipes_expanded.dropna()
df_recipes_expanded

,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,...,rating_mean,rating_count,all_reviews,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbohydrates_pdv
0,arriba baked winter squash mexican style,137739.0,55.0,47892.0,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...",11.0,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",...,5.0,3.0,[ I used an acorn squash and recipe#137681 Swe...,51.5,0.0,13.0,0.0,2.0,0.0,4.0
130923,marinated greek feta,141627.0,5.0,197023.0,2005-10-17,"['15-minutes-or-less', 'time-to-make', 'course...",4.0,['cut the feta cheese into 2cm cubes and place...,"i found this simple, super-quick-to-make recip...","['greek feta cheese', 'olive oil', 'cracked bl...",...,5.0,8.0,[Just delicious and a wonderful way to keep my...,280.9,18.0,39.0,23.0,36.0,22.0,9.0
130921,marinated goat cheese with fresh oregano,63342.0,65.0,56081.0,2003-05-28,"['weeknight', 'time-to-make', 'course', 'main-...",2.0,"['place oil , oregano and pepper in a narrow s...",another one from the atkins low carb cook book...,"['extra virgin olive oil', 'oregano leaves', '...",...,5.0,1.0,[This was really good! I wasn't sure if I lik...,284.7,16.0,10.0,24.0,46.0,9.0,7.0
130920,marinated goat cheese spread dip,258115.0,195.0,444132.0,2007-10-10,"['time-to-make', 'course', 'main-ingredient', ...",6.0,['arrange cheese in a shallow dish in a single...,a decedent spread that wins raves at potlucks ...,"['soft fresh goat cheese', 'kalamata olive', '...",...,5.0,3.0,"[I'm not a big fan of goat cheese, but made th...",555.9,58.0,13.0,56.0,83.0,73.0,4.0
130917,marinated goat cheese,56514.0,2885.0,62043.0,2003-03-18,"['weeknight', 'time-to-make', 'course', 'main-...",4.0,['cut goat cheese into 1 cm rounds and place i...,this recipe is from the ottawa citizen. this i...,"['chevre cheese', 'dried chili', 'whole black ...",...,5.0,2.0,[This is a delicious appetizer to have on hand...,325.6,13.0,10.0,27.0,13.0,27.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2673,affordable basic restaurant style tomato and m...,115199.0,15.0,118159.0,2005-04-04,"['15-minutes-or-less', 'time-to-make', 'course...",6.0,"['in a medium saucepan', 'bring salted water ,...",ever had this comfort soup in a restaurant? t...,"['tomato juice', 'water', 'onion', 'macaroni',...",...,3.9,10.0,"[hello Andre, I have finally come on to rate ...",291.2,26.0,73.0,7.0,13.0,59.0,9.0
174846,roast pork pernil puerto rican style,316839.0,255.0,812387.0,2008-07-31,"['main-ingredient', 'cuisine', 'preparation', ...",10.0,['the night before peel garlic and with a piln...,"tasty roast pork, puerto rican style, marine o...","['pork shoulder', 'corn oil', 'white vinegar',...",...,3.9,10.0,[Foolproof recipe for making a flavorful and m...,1980.0,301.0,8.0,172.0,70.0,591.0,5.0
132386,mean green juice for juicer,492690.0,17.0,442988.0,2012-12-31,"['30-minutes-or-less', 'time-to-make', 'course...",2.0,"['wash all ingredients thoroughly', 'run throu...","this is the recipe for ""mean green"" juice used...","['cucumber', 'celery ribs', 'apples', 'kale le...",...,3.9,10.0,[DH and I are juicing and this is a great go t...,499.2,32.0,17.0,28.0,37.0,61.0,20.0
62781,creamy cocoa mix extra large batch,107668.0,15.0,154044.0,2005-01-05,"['15-minutes-or-less', 'time-to-make', 'course...",3.0,['mix all ingredients in a large bowl and blen...,"i like this cocoa for many reasons, only one o...","['unsweetened cocoa powder', 'powdered sugar',...",...,3.9,10.0,"[I am feeling under-the-weather today, and nee...",1511.8,140.0,210.0,232.0,177.0,151.0,26.0


In [30]:
df_recipes_expanded.to_csv("../data/enriched_cleaned_recipes.csv")

In [31]:
def get_unique_tags(df):
    """
    Extracts all unique tags from the tags column.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing the tags column
    
    Returns:
    list: Sorted list of all unique tags
    dict: Dictionary with count of each tag's occurrence
    """
    # Convert string representations of lists to actual lists if needed
    if isinstance(df['tags'].iloc[0], str):
        tags_lists = df['tags'].apply(ast.literal_eval)
    else:
        tags_lists = df['tags']
    
    # Flatten the list of lists and get unique tags
    all_tags = []
    for tags in tags_lists:
        all_tags.extend(tags)
    
    # Create a dictionary with tag counts
    tag_counts = {}
    for tag in all_tags:
        tag_counts[tag] = tag_counts.get(tag, 0) + 1
    
    # Sort tags by frequency (most common first)
    sorted_tag_counts = dict(sorted(tag_counts.items(), 
                                  key=lambda x: x[1], 
                                  reverse=True))
    
    return list(sorted_tag_counts.keys()), sorted_tag_counts

In [32]:
# Get unique tags and their counts
unique_tags, tag_counts = get_unique_tags(df_recipes_expanded)

# Print total number of unique tags
print(f"Total number of unique tags: {len(unique_tags)}")

# Print the 50 most common tags and their counts
print("\n50 most common tags:")
for tag, count in list(tag_counts.items())[:50]:
    print(f"{tag}: {count} recipes")

# If you want to find recipes with a specific tag
def find_recipes_with_tag(df, target_tag):
    if isinstance(df['tags'].iloc[0], str):
        return df[df['tags'].apply(lambda x: target_tag in ast.literal_eval(x))]
    return df[df['tags'].apply(lambda x: target_tag in x)]

# Example: Find recipes tagged as 'vegetarian'
# vegetarian_recipes = find_recipes_with_tag(df_recipes, 'vegetarian')

Total number of unique tags: 542

50 most common tags:
preparation: 160108 recipes
time-to-make: 156472 recipes
course: 151638 recipes
main-ingredient: 118760 recipes
dietary: 113434 recipes
easy: 88782 recipes
occasion: 79256 recipes
cuisine: 62952 recipes
low-in-something: 59620 recipes
main-dish: 49651 recipes
60-minutes-or-less: 48570 recipes
equipment: 48182 recipes
number-of-servings: 40381 recipes
meat: 39716 recipes
30-minutes-or-less: 38601 recipes
vegetables: 36701 recipes
taste-mood: 36335 recipes
4-hours-or-less: 34091 recipes
north-american: 33552 recipes
3-steps-or-less: 31982 recipes
15-minutes-or-less: 30644 recipes
low-sodium: 30068 recipes
low-carb: 29764 recipes
desserts: 29329 recipes
healthy: 27123 recipes
dinner-party: 26673 recipes
beginner-cook: 25319 recipes
low-calorie: 25132 recipes
5-ingredients-or-less: 25042 recipes
low-cholesterol: 24967 recipes
holiday-event: 24227 recipes
vegetarian: 23715 recipes
low-protein: 22596 recipes
inexpensive: 22463 recipes
fr

In [33]:
unique_tags

['preparation',
 'time-to-make',
 'course',
 'main-ingredient',
 'dietary',
 'easy',
 'occasion',
 'cuisine',
 'low-in-something',
 'main-dish',
 '60-minutes-or-less',
 'equipment',
 'number-of-servings',
 'meat',
 '30-minutes-or-less',
 'vegetables',
 'taste-mood',
 '4-hours-or-less',
 'north-american',
 '3-steps-or-less',
 '15-minutes-or-less',
 'low-sodium',
 'low-carb',
 'desserts',
 'healthy',
 'dinner-party',
 'beginner-cook',
 'low-calorie',
 '5-ingredients-or-less',
 'low-cholesterol',
 'holiday-event',
 'vegetarian',
 'low-protein',
 'inexpensive',
 'fruit',
 'american',
 'eggs-dairy',
 'oven',
 'low-saturated-fat',
 'side-dishes',
 'kid-friendly',
 'pasta-rice-and-grains',
 'comfort-food',
 'healthy-2',
 'european',
 'presentation',
 'poultry',
 'for-1-or-2',
 'lunch',
 'stove-top',
 'seasonal',
 'low-fat',
 'chicken',
 'weeknight',
 'appetizers',
 'brunch',
 'to-go',
 'for-large-groups',
 'beef',
 'one-dish-meal',
 'cheese',
 'salads',
 'seafood',
 'breads',
 'breakfast',
 '

In [34]:
df_recipes_expanded.columns

Index(['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags',
       'n_steps', 'steps', 'description', 'ingredients', 'n_ingredients',
       'rating_mean', 'rating_count', 'all_reviews', 'calories',
       'total_fat_pdv', 'sugar_pdv', 'sodium_pdv', 'protein_pdv',
       'saturated_fat_pdv', 'carbohydrates_pdv'],
      dtype='object')

In [35]:
sampled_df = df_recipes_expanded.sample(1000)

In [36]:
df = sampled_df


def combine_fields(row):
    tags = row['tags'] if isinstance(row['tags'], str) else ', '.join(eval(row['tags']))
    ingredients = row['ingredients'] if isinstance(row['ingredients'], str) else ', '.join(eval(row['ingredients']))
    description = row['description'] if pd.notnull(row['description']) else ""
    return f"{row['name']}. Tags: {tags}. Ingredientes: {ingredients}. Descrição: {description}"

df['combined_text'] = df.apply(combine_fields, axis=1)

model = SentenceTransformer('all-MiniLM-L6-v2')

start_time = time.time()

batch_size = 256
texts = df['combined_text'].tolist()
embeddings = model.encode(texts, batch_size=batch_size, show_progress_bar=True)

end_time = time.time()
print(f"Time to compute embeddings: {end_time - start_time:.2f} seconds")

embeddings = np.array(embeddings).astype('float32')
embedding_dim = embeddings.shape[1]

index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings)

print(f"Total recipes indexed: {index.ntotal}")




Batches: 100%|██████████| 4/4 [00:14<00:00,  3.63s/it]

Time to compute embeddings: 14.60 seconds
Total recipes indexed: 1000


In [37]:
def search_recipes(query, k=5):
    query_embedding = model.encode([query]).astype('float32')
    distances, indices = index.search(query_embedding, k)
    results = df.iloc[indices[0]]
    return results[['name', 'tags', 'description', 'ingredients']], distances[0]

query = "high fat delicious dinner"
results, distances = search_recipes(query, k=5)
print("Resultados da busca:")
print(results)
print("Distâncias:", distances)

Resultados da busca:
                                              name  \
159697    pilaf with chicken  spinach  and walnuts   
87822                           fried potato salad   
187929  sizzling chilli prawns with egg fried rice   
180253                     sarasota s shrimp toast   
185456                   shrimp and egg fried rice   

                                                     tags  \
159697  ['60-minutes-or-less', 'time-to-make', 'course...   
87822   ['time-to-make', 'main-ingredient', 'preparati...   
187929  ['60-minutes-or-less', 'time-to-make', 'course...   
180253  ['weeknight', '60-minutes-or-less', 'time-to-m...   
185456  ['30-minutes-or-less', 'time-to-make', 'course...   

                                              description  \
159697  use leftover meat from a roasted chicken, or p...   
87822   from diners, drive-ins and dives cookbook. cou...   
187929  most of this recipe can be prepared ahead of t...   
180253  this is a great appetizer, but this

In [41]:
query = "low sugar and low carbs breakfast for two"
results, distances = search_recipes(query, k=5)
print("Resultados da busca:")
print(results)
print("Distâncias:", distances)

Resultados da busca:
                                                     name  \
5111                 amy s low fat blueberry bran muffins   
103831  healthy summer time  or anytime  omelet sammie...   
28662                                  breakfast surprise   
126909     low carb chili   you never know the difference   
16624                      banana instant oatmeal muffins   

                                                     tags  \
5111    ['60-minutes-or-less', 'time-to-make', 'course...   
103831  ['15-minutes-or-less', 'time-to-make', 'course...   
28662   ['15-minutes-or-less', 'time-to-make', 'course...   
126909  ['lactose', 'time-to-make', 'course', 'prepara...   
16624   ['30-minutes-or-less', 'time-to-make', 'course...   

                                              description  \
5111    i found this recipe online a long time ago and...   
103831  this is a standby breakfast for dh and i in th...   
28662   a great little breakfast or snack from the org...   
1